# Design of a PV powerplant

We want to install photovoltaic panels on the south part of the roof of the thermodynamics laboratory. The roof has a surface area of 300 m². A supplier has provided an offer of 350 € for each PV panel with the following characteristics:

<table>
<tr>
<td>$V_{oc} \, [V]$</td>
<td>$I_{sc} \, [A]$</td>
<td>$V_m \, [V]$</td>
<td>$I_{m} \, [A]$</td>
<td>$\Delta V_{oc}/ \Delta T \, [\% \, °C^{-1}]$</td>
<td>$\Delta V_{m}/ \Delta T \, [\% \, °C^{-1}]$</td>
<td>$NOCT \, [°C]$</td>
<td>$S \, [m^2]$</td>
</tr>
<tr>
<td>$67.5$</td>
<td>$3.75$</td>
<td>$54.8$</td>
<td>$3.47$</td>
<td>$-0.3$</td>
<td>$-0.4$</td>
<td>$45$</td>
<td>$1.32$</td>
</tr>
</table>

He has also proposed a offer of 3180 € for each inverter with the following characteristics:

<table>
<tr>
<td>$Inventer \, [-]$</td>
<td>$P_{nom} \, [kW]$</td>
<td>$P_{max} \, [kW]$</td>
<td>$V_{in,dc} \, [V]$</td>
<td>$V_{in,{mppt}} \, [V]$</td>
<td>$V_{ac,{out}} \, [V]$</td>
<td>$I_{ac,{out}} \, [A]$</td>
</tr>
<tr>
<td>$1$</td>
<td>$10.6$</td>
<td>$11.7$</td>
<td>$600$</td>
<td>Between 180 & 450 V</td>
<td>$400$</td>
<td>$28.6$</td>
</tr>
</table>

Considering that the installation cost and the cost due to the connection to the electricity grid are 0.9 €/$W_{peak,installed}$ , what is the total installation cost?

What is the return on investment? Consider an electricity price of 85 €/MWh and that the green certificates are sold for 65 € (We receive 5 green certificates per MWh)



# How to solve?

- Calculation of the solar potential with the simple method. (tilt angle = $30°C$, gamma = 45°C° , L=-5,55° , $\phi = 50,58°$ )

- Calculation of the electrical energy produced by each PV panel over one year (in kWh/kWp) (consider a performance factor of 75 %)

- Calculation of the maximum and minimum PV voltage.

- Determination of the optimal number of PV panels placed on the roof.

- Calculation of the total construction cost, the cash inflow and the return on investment.

In [2]:
from numpy import *
S_labo = 300
S_p = 1.32
V_m = 54.8
I_m = 3.47
I_std = 1000
k = 0.75
G_i_gamma = 1024

N_p = S_labo/S_p

P_i = N_p*V_m*I_m

eta_n = V_m*I_m/(I_std*S_p)

E_e = S_p*G_i_gamma*eta_n*k

E_s = E_e/(V_m*I_m/1000)

print("P_i =", P_i, "kW", "," , "eta_n =", eta_n, "%", "," , "E_e =", E_e, "kWh/year", "," , "E_s =", E_s, "kWh/kW")

P_i = 43217.27272727272 kW , eta_n = 0.14405757575757577 % , E_e = 146.03980800000002 kWh/year , E_s = 768.0 kWh/kW


In [3]:
T_a_min = -10
T_a_max = 38
NOCT = 45
G_min = 0
G_max = 1000
V_oc = 67.5
V_m = 54.8
DV_oc_DT = -0.003
DV_m_DT = -0.004


T_c_min = T_a_min + ((NOCT - 20)*G_min/800)
T_c_max = T_a_max + ((NOCT - 20)*G_max/800)

V_oc_max = V_oc * (1+((T_c_min-25)*DV_oc_DT))
V_m_min = V_m * (1+((T_c_max-25)*DV_m_DT))

print "T_c_min =", T_c_min, "°C", "," , "T_c_max =", T_c_max, "°C", "," , "V_oc_max =", V_oc_max, "V", "," , "V_m_min =", V_m_min, "V"

T_c_min = -10 °C , T_c_max = 69 °C , V_oc_max = 74.5875 V , V_m_min = 45.1552 V


In [7]:
from numpy import *
V_in_MPPT_max = 450
V_oc_max = 74.5875
V_in_MPPT_min = 180
V_m_min = 45.1552
P_nom_onduleur = 10600
N_p_max = int(floor(300/1.32))
V_m = 54.8
I_m = 3.47

N_p_max_serie = int(floor(V_in_MPPT_max/V_oc_max))
N_p_min_serie = int(ceil(V_in_MPPT_min/V_m_min))
N_p_serie = 5
N_p_parallele = int(floor(P_nom_onduleur/(N_p_serie*190.16)))
n_onduleur = int(floor(N_p_max/(N_p_parallele*N_p_serie)))
N_p_total = N_p_parallele*N_p_serie*n_onduleur
P_totale = N_p_total*I_m*V_m
E_e = 950*P_totale/1000

C_total = N_p_total*350+n_onduleur*3180+0.9*P_totale
Gain = 0.085*E_e + 5*65*E_e/1000
PaybackTimeAnnees = int(floor(C_total/Gain))
PaybackTimeMois = int((C_total/Gain - int(floor(C_total/Gain)))*12)


print "N_p_max_series =", N_p_max_serie, "," , "N_p_min_series =" , N_p_min_serie, "," , "N_p_series =" ,N_p_serie, "," , "N_p_parallel =" ,N_p_parallele, "," , "Total Cost =", C_total , "," , "Cash inflow =", Gain, "," , "Investment Payback Time =", PaybackTimeAnnees ,"years and", PaybackTimeMois, "month"  

N_p_max_series = 6 , N_p_min_series = 4 , N_p_series = 5 , N_p_parallel = 11 , Total Cost = 127370.888 , Cash inflow = 16294.46764 , Investment Payback Time = 7 years and 9 month
